In [1]:
import pandas as pd
import itertools
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer

In [2]:
data1 = pd.read_csv('Downloads/GBdata.csv')
data1.shape

(355, 157)

In [3]:
data_2 = pd.read_excel('Downloads/Guglielmetti.xlsx')

In [4]:
data_2.to_csv('/Users/ltemereva/Downloads/Guglielmetti.csv')

In [5]:
data2 = pd.read_csv('Downloads/Guglielmetti.csv')

In [6]:
del data2['TimeConvB']

In [7]:
dd = data2.drop(data2.columns[0], axis=1)
dd.shape

(54, 157)

In [8]:
data_3 = pd.read_excel('Downloads/Fox.xlsx')

In [9]:
data_3.to_csv('/Users/ltemereva/Downloads/Fox.csv')

In [10]:
data3 = pd.read_csv('Downloads/Fox.csv')
del data3['TimeConvB']
ddd = data3.drop(data3.columns[0], axis=1)
ddd.shape

(26, 157)

In [11]:
data_cl = pd.read_csv('Downloads/clofazimine_Brazil_English_IPD_format.csv')
data_comb = pd.concat([dd, ddd,data_cl], sort=False)
data_comb.shape

(3428, 157)

In [12]:
data_comb2 = data_comb

In [173]:
data4 = pd.concat([data1, data_comb2], sort=False).reset_index(drop=True)
data4['AIntNo'] = data4['AIntNo'].map(lambda x: np.nan if x >= 888 else x)

In [174]:
data4.shape

(3783, 157)

In [252]:
data4.to_csv('~/Desktop/data4.csv')

In [15]:
resist = data4.iloc[:,20:41].apply(pd.value_counts)
resist.to_csv('/Users/ltemereva/resist.csv')
resist.T

,R,S,U
DSTH,3702.0,81.0,NaN
DSTR,3781.0,2.0,NaN
DSTE,1483.0,2092.0,208.0
DSTZ,1271.0,1101.0,1411.0
DSTS,1717.0,1819.0,247.0
DSTRfb,120.0,26.0,289.0
DSTAm,140.0,323.0,3320.0
DSTCm,131.0,242.0,3410.0
DSTKm,162.0,149.0,3472.0
DSTOfx,248.0,241.0,3294.0


In [251]:
import statsmodels.imputation.mice as mice
import statsmodels.api as sm
imputed_variables = ((data4['DSTCfx']) & (data4['DSTLfx']) & (data4['DSTMfx']) & (data4['DSTOfx']) & (data4['DSTAm']) & (data4['DSTCm']) & (data4['DSTKm']) & (data4['DSTS']), axis=1)
imp = mice.MICEData(full)
imp.update_all(10)
imp.data

SyntaxError: invalid syntax (<ipython-input-251-4ef91accf25e>, line 3)

In [230]:
# main variables as numbers

used_drugs = data4.iloc[:,pd.np.r_[105:134]].applymap(lambda x: 1 if x == 'Yes' else -1 if x == 'No' else 0)
#treat_time = data4['ATotalDur']
main = pd.concat([used_drugs], axis=1)

# resistances as boolean series

res_injectables = (data4['DSTAm'] == 'R') | (data4['DSTCm'] == 'R') | (data4['DSTKm'] == 'R') | (data4['DSTS'] == 'R')
res_fq = (data4['DSTCfx'] == 'R') | (data4['DSTLfx'] == 'R') | (data4['DSTMfx'] == 'R') | (data4['DSTOfx'] == 'R')
res_first_line = (data4['DSTH'] == 'R') & (data4['DSTR'] == 'R')

res_Cfx = (data4['DSTCfx'] == 'R') & (data4['DSTLfx'] != 'R') & (data4['DSTMfx'] != 'R') & (data4['DSTOfx'] != 'R') 
res_Mfx = (data4['DSTMfx'] == 'R') & (data4['DSTLfx'] != 'R') & (data4['DSTCfx'] != 'R') & (data4['DSTOfx'] != 'R')
res_Lfx = (data4['DSTLfx'] == 'R') & (data4['DSTCfx'] != 'R') & (data4['DSTMfx'] != 'R') & (data4['DSTOfx'] != 'R')
res_Ofx = (data4['DSTOfx'] == 'R') & (data4['DSTLfx'] != 'R') & (data4['DSTMfx'] != 'R') & (data4['DSTCfx'] != 'R')
res_Am = (data4['DSTAm'] == 'R') & (data4['DSTKm'] != 'R') & (data4['DSTCm'] != 'R') & (data4['DSTS'] != 'R')
res_Cm = (data4['DSTCm'] == 'R') & (data4['DSTKm'] != 'R') & (data4['DSTAm'] != 'R') & (data4['DSTS'] != 'R')
res_Km = (data4['DSTKm'] == 'R') & (data4['DSTAm'] != 'R') & (data4['DSTCm'] != 'R') & (data4['DSTS'] != 'R')
res_S = (data4['DSTS'] == 'R') & (data4['DSTKm'] != 'R') & (data4['DSTCm'] != 'R') & (data4['DSTAm'] != 'R')

res_first_line_only = res_first_line & ~res_injectables & ~res_fq
res_Cfx_only = res_Cfx & res_first_line & ~res_injectables # for checking numbers of
res_one_fq = (res_Cfx | res_Mfx | res_Lfx | res_Ofx) & res_first_line & ~res_injectables
res_one_injectables = (res_Am | res_Cm | res_Km | res_S) & res_first_line & ~res_fq
res_injectables_only = res_first_line & res_injectables & ~res_fq
res_fq_only = res_first_line & res_fq & ~res_injectables
res_fq_injectables = res_injectables & res_fq & res_first_line

# covariates as numbers 

age = data4['Age'].fillna(data4['Age'].mean())# todo imput with average (перевести в интервалы?)
sex_m = data4['Sex'].map(lambda x: 1 if x[0] == 'F' else -1 if x == 'M' else np.nan) 
smoking_m = data4['Smoking'].map(lambda x: 1 if x == 'Yes' else -1 if x == 'No' else np.nan)
art_m = data4['ART'].map(lambda x: 1 if x == 'Yes' else -1 if x == 'No' else np.nan)
hiv_m = data4['HIV'].map({'Neg': -1, 'Pos': 1, 'U': np.nan})
pasttx_m = data4['PastTx'].map(lambda x: 1 if x == 'Yes' else -1 if x == 'No' else np.nan)
pasttype_m = data4['PastType'].map({'FLD': 1, 'SLD': 2, 'U': np.nan})
site_m = data4['Site'].map(lambda x: 1 if x == 'Pulm' else 2 if x == 'Extrapulm' else 3 if x=='Both' else np.nan)
aintno = data4['AIntNo'].fillna(data4['AIntNo'].mean())
acontno = data4['AContNo'].fillna(data4['AIntNo'].mean()) 
atotaldur = data4['ATotalDur'].fillna(data4['ATotalDur'].mean())

# resistances as numbers
res_first_line_m = res_first_line.map({True: 1, False: 0}).rename('FirstLine')
res_first_line_only_m = res_first_line_only.map({True: 1, False: 0}).rename('FirstLineOnly')
res_one_injectables_m = res_one_injectables.map({True: 1, False: 0}).rename('OneInj')
res_one_fq_m = res_one_fq.map({True: 1, False: 0}).rename('OneFq')
res_injectables_only_m = res_injectables_only.map({True: 1, False: 0}).rename('InjOnly')
res_fq_only_m = res_fq_only.map({True: 1, False: 0}).rename('FqOnly')
res_fq_injectables_m = res_fq_injectables.map({True: 1, False: 0}).rename('FqInjectables')

# outcomes as numbers

#outcomes_m = data4['Outcome'].map({'Cure': -1, 'Complete': -1, 'Death': 1, 'Lost': 1, 'Fail': 1, 'Transfer': 1})
#relapse_m = data4['Relapse'].map({'Yes': 1, 'No': 0, 'U': 0})
out_rel = data4[['Outcome', 'Relapse']]
outcomes_m = out_rel.apply(lambda x: 1 if (x[0] == 'Death' or x[0] == 'Lost' or x[0] == 'Fail' or x[0] == 'Transfer' or x[1] == 'Yes') else -1, axis = 1).rename('Outcome')

#covariates = pd.concat([age, sex_m, hiv_m, pasttx_m, aintno, acontno, atotaldur], axis=1)
covariates = pd.concat([age, sex_m, smoking_m, art_m, hiv_m, pasttx_m, pasttype_m, site_m, data4['AIntNo'], data4['AContNo'], data4['ATotalDur']], axis=1)
#covariates.isna().sum() / covariates.shape[0] * 100
covariates.describe()

,Age,Sex,Smoking,ART,HIV,PastTx,PastType,Site,AIntNo,AContNo,ATotalDur
count,3783.000000,3782.000000,3415.000000,21.000000,3760.000000,3664.000000,2836.000000,3783.000000,426.000000,305.000000,426.000000
mean,38.634999,-0.278160,-0.941435,0.904762,-0.857447,0.928493,1.978491,1.048903,6.427230,4.944262,20.924131
std,12.897160,0.960662,0.337244,0.436436,0.514641,0.371399,0.145100,0.295369,1.720632,1.543292,8.285887
min,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,-1.000000,-1.000000,1.000000,-1.000000,1.000000,2.000000,1.000000,5.000000,4.000000,18.000000
50%,38.000000,-1.000000,-1.000000,1.000000,-1.000000,1.000000,2.000000,1.000000,6.000000,5.000000,20.300000
75%,47.000000,1.000000,-1.000000,1.000000,-1.000000,1.000000,2.000000,1.000000,7.000000,6.000000,24.000000
max,93.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,12.000000,10.000000,60.000000


In [19]:
desc = covariates.describe()
desc.to_csv('/Users/ltemereva/desc.csv')


In [231]:
variables = pd.concat([res_first_line_m, res_first_line_only_m, res_one_injectables_m, res_one_fq_m, res_injectables_only_m, res_fq_only_m, res_fq_injectables_m], axis=1)
variables.describe()

,FirstLine,FirstLineOnly,OneInj,OneFq,InjOnly,FqOnly,FqInjectables
count,3783.000000,3783.000000,3783.000000,3783.000000,3783.000000,3783.000000,3783.000000
mean,0.978324,0.512027,0.378271,0.008459,0.396511,0.008723,0.061063
std,0.145642,0.499921,0.485020,0.091595,0.489237,0.093002,0.239477
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [216]:
percents = covariates.notna().sum() / covariates.shape[0] * 100
percents.to_csv('/Users/ltemereva/Desktop/perc.csv')
percents
#covariates['Sex'].value_counts()

/Users/ltemereva/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


Age          100.000000
Sex           99.973566
Smoking       90.272271
ART            0.555115
HIV           99.392017
PastTx        96.854348
PastType      74.966957
Site         100.000000
AIntNo        11.260904
AContNo        8.062384
ATotalDur     11.260904
dtype: float64

In [197]:
data4[res_first_line]['ATotalDur'].mean()

20.95689411764705

In [103]:
res_fq_only.value_counts()

False    3750
True       33
dtype: int64

In [391]:
# Q1: injectables only vs first-line only

q1 = pd.concat([res_injectables_only_m, res_first_line_only_m], axis=1)
Xf = pd.concat([covariates, q1], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates


array([1.0025961 , 0.73425153, 0.85104479, 1.38040891, 0.99992393,
       1.01775608, 0.90575952, 0.90137272, 0.67457295])

In [438]:
# Experiment 1.1: MDR only (with covariates and drugs)

MDR = pd.concat([res_first_line_only_m], axis=1)
Xf = pd.concat([covariates, MDR], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values
lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([1.00235227, 0.73329494, 0.85647698, 1.39411058, 0.99993483,
       1.01874036, 0.90601909, 0.70155284])

In [439]:
from sklearn.metrics import roc_auc_score
y_pred = lr.predict(X)

roc_auc_score(y, y_pred)

0.5985074626865671

In [455]:
# Experiment 1.2: MDR only (without covariates)
MDR = pd.concat([res_first_line_only_m], axis=1)
Xf = pd.concat([MDR], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([1.07524481])

In [442]:
# Experiment 2.1: MDR + one fq only (with covariates)
one_fq = pd.concat([res_one_fq_m], axis=1)
Xf = pd.concat([covariates, one_fq], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([1.00179132, 0.72704031, 0.84015094, 1.42755349, 0.99997306,
       1.01835166, 0.90594253, 1.2716924 ])

In [444]:
# Experiment 2.2: MDR + one fq only (without covariates and drugs)
one_fq = pd.concat([res_one_fq_m], axis=1)
Xf = pd.concat([one_fq], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([1.35025845])

In [423]:
# Experiment 3.1: MDR + all fq (with covariates and drugs)
fq = pd.concat([res_fq_only_m], axis=1)
Xf = pd.concat([covariates, fq], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates
lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([1.00179132, 0.72704031, 0.84015094, 1.42755349, 0.99997306,
       1.01835166, 0.90594253, 1.2716924 ])

In [446]:
# Experiment 3.2: MDR + all fq (without covariates and drugs)
fq = pd.concat([res_fq_only_m], axis=1)
Xf = pd.concat([fq], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([1.44532338])

In [450]:
# Experiment 4.1: MDR + all fq and all injectables (with covariates and drugs)
fq_inj = pd.concat([res_fq_injectables_m], axis=1)
Xf = pd.concat([covariates, fq_inj], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([1.00181439, 0.73004486, 0.85153483, 1.39361707, 0.99993213,
       1.01758086, 0.90505556, 1.18460922])

In [452]:
# Experiment 4.2: MDR + all fq and all injectables (without covariates and drugs)
fq_inj = pd.concat([res_fq_injectables_m], axis=1)
Xf = pd.concat([fq_inj], axis=1)
full = pd.concat([Xf, outcomes_m], axis=1).dropna()
Xd = full.drop(['Outcome'], axis = 1)
yd = full['Outcome']
X = Xd.values
y = yd.values

lr = LogisticRegression()
lr.fit(X, y)
odd_rates = np.exp(lr.coef_)[0] # odds rates
odd_rates

array([0.61505234])

In [222]:
import statsmodels.imputation.mice as mice
import statsmodels.api as sm
full = pd.concat([covariates, variables, outcomes_m], axis=1)
imp = mice.MICEData(full)
imp.update_all(10)
imp.data

/Users/ltemereva/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Age,Sex,Smoking,ART,HIV,PastTx,PastType,Site,AIntNo,AContNo,ATotalDur,FirstLine,FirstLineOnly,OneInj,OneFq,InjOnly,FqOnly,FqInjectables,Outcome
0,70.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1,5.0,4.0,18.00,1,0,0,0,1,0,0,1
1,67.0,1.0,-1.0,-1.0,-1.0,1.0,2.0,1,4.0,3.0,18.00,1,1,0,0,0,0,0,-1
2,29.3,1.0,-1.0,1.0,-1.0,-1.0,1.0,1,5.0,4.0,17.90,1,1,0,0,0,0,0,-1
3,25.3,-1.0,-1.0,1.0,-1.0,1.0,2.0,1,5.0,4.0,18.30,1,0,1,0,1,0,0,-1
4,44.6,1.0,-1.0,1.0,-1.0,-1.0,1.0,1,7.0,6.0,14.40,1,0,1,0,1,0,0,-1
5,25.5,1.0,1.0,1.0,-1.0,-1.0,2.0,1,6.0,5.0,18.30,1,0,0,0,0,0,1,-1
6,26.6,-1.0,-1.0,1.0,-1.0,-1.0,2.0,3,3.0,2.0,2.70,1,1,0,0,0,0,0,1
7,41.1,-1.0,1.0,1.0,-1.0,-1.0,2.0,1,5.0,4.0,18.70,1,0,0,0,0,0,1,-1
8,32.1,1.0,-1.0,1.0,-1.0,-1.0,1.0,1,4.0,3.0,18.20,1,1,0,0,0,0,0,-1
9,22.6,1.0,-1.0,1.0,-1.0,-1.0,1.0,1,5.0,4.0,18.50,1,1,0,0,0,0,0,-1


In [225]:
import statsmodels.imputation.mice as mice
import statsmodels.api as sm
full = pd.concat([covariates, variables, outcomes_m], axis=1)
imp = mice.MICEData(full)
fml = 'Outcome ~ FirstLineOnly'
mice = mice.MICE(fml, sm.GLM, imp)
results = mice.fit(10, 10)
results.summary()

/Users/ltemereva/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: MICE
=====================================================================
Method:                     MICE          Sample size:           3783
Model:                      GLM           Scale                  0.88
Dependent variable:         Outcome       Num. imputations       10  
---------------------------------------------------------------------
               Coef.  Std.Err.    t     P>|t|   [0.025  0.975]  FMI  
---------------------------------------------------------------------
Intercept     -0.3608   0.0219 -16.5060 0.0000 -0.4036 -0.3179 0.0000
FirstLineOnly  0.0330   0.0305   1.0788 0.2807 -0.0269  0.0928 0.0000
=====================================================================

"""

In [228]:
np.exp(results.params[1:])

array([1.03350251])

In [240]:
import statsmodels.imputation.mice as mice
import statsmodels.api as sm
full = pd.concat([covariates, variables, outcomes_m], axis=1)
imp = mice.MICEData(full)
fml = 'Outcome ~ FirstLineOnly + Age + ART + Sex + Smoking + HIV + PastTx + PastType + AIntNo + AContNo + ATotalDur'
mice = mice.MICE(fml, sm.GLM, imp)
results = mice.fit(10, 10)
results.summary()

/Users/ltemereva/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: MICE
====================================================================
Method:                    MICE          Sample size:           3783
Model:                     GLM           Scale                  0.76
Dependent variable:        Outcome       Num. imputations       10  
--------------------------------------------------------------------
               Coef.  Std.Err.    t    P>|t|   [0.025  0.975]  FMI  
--------------------------------------------------------------------
Intercept      0.7459   0.2720  2.7417 0.0061  0.2127  1.2791 0.5435
FirstLineOnly  0.0371   0.0744  0.4984 0.6182 -0.1087  0.1828 0.8448
Age           -0.0065   0.0017 -3.7014 0.0002 -0.0099 -0.0030 0.5787
ART            0.0050   0.1475  0.0339 0.9729 -0.2841  0.2941 0.8094
Sex           -0.0354   0.0280 -1.2622 0.2069 -0.0904  0.0196 0.7069
Smoking        0.0353   0.0536  0.6589 0.5099 -0.0698  0.1405 0.6292
HIV            0.0737   0.0523  1.4099 0.1586 -0.0288  0.1763 0.7017
PastTx         0.2109   0.0516  4.0844 0.0000  0.1097  0.3120 0.2349
PastType      -0.0162   0.1054 -0.1532 0.8782 -0.2228  0.1905 0.3697
AIntNo         0.0586   0.0714  0.8213 0.4115 -0.0813  0.1986 0.9548
AContNo       -0.1488   0.0917 -1.6232 0.1045 -0.3285  0.0309 0.9688
ATotalDur     -0.0326   0.0082 -3.9806 0.0001 -0.0486 -0.0165 0.9533
====================================================================

"""

In [235]:
np.exp(results.params[1:])

array([1.02101439, 0.99363559, 0.97186634, 1.02885086, 0.96297427,
       1.07218061, 1.21841   , 1.03371165, 0.99646579, 1.02669675,
       0.92304671, 0.96751451])

In [244]:
import statsmodels.imputation.mice as mice
import statsmodels.api as sm
full = pd.concat([covariates, variables, outcomes_m], axis=1)
imp = mice.MICEData(full)
fml = 'Outcome ~ OneFq'
mice = mice.MICE(fml, sm.GLM, imp)
results = mice.fit(10, 10)
results.summary()

/Users/ltemereva/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: MICE
=================================================================
Method:                   MICE         Sample size:          3783
Model:                    GLM          Scale                 0.88
Dependent variable:       Outcome      Num. imputations      10  
-----------------------------------------------------------------
           Coef.  Std.Err.    t     P>|t|   [0.025  0.975]  FMI  
-----------------------------------------------------------------
Intercept -0.3452   0.0153 -22.5147 0.0000 -0.3753 -0.3152 0.0000
OneFq      0.1577   0.1667   0.9461 0.3441 -0.1690  0.4845 0.0000
=================================================================

"""

In [245]:
np.exp(results.params[1:])

array([1.17086322])

In [246]:
import statsmodels.imputation.mice as mice
import statsmodels.api as sm
full = pd.concat([covariates, variables, outcomes_m], axis=1)
imp = mice.MICEData(full)
fml = 'Outcome ~ OneFq + Age + ART + Sex + Smoking + HIV + PastTx + PastType + AIntNo + AContNo + ATotalDur'
mice = mice.MICE(fml, sm.GLM, imp)
results = mice.fit(10, 10)
results.summary()

/Users/ltemereva/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: MICE
================================================================
Method:                  MICE         Sample size:          3783
Model:                   GLM          Scale                 0.77
Dependent variable:      Outcome      Num. imputations      10  
----------------------------------------------------------------
           Coef.  Std.Err.    t    P>|t|   [0.025  0.975]  FMI  
----------------------------------------------------------------
Intercept  0.6432   0.3486  1.8451 0.0650 -0.0400  1.3265 0.7318
OneFq      0.5101   0.1966  2.5943 0.0095  0.1247  0.8956 0.3482
Age       -0.0068   0.0019 -3.5218 0.0004 -0.0106 -0.0030 0.6508
ART        0.0118   0.0699  0.1686 0.8661 -0.1252  0.1488 0.6597
Sex       -0.0295   0.0196 -1.5088 0.1313 -0.0679  0.0088 0.3934
Smoking    0.0431   0.0416  1.0363 0.3001 -0.0384  0.1246 0.3864
HIV        0.0701   0.0469  1.4941 0.1351 -0.0218  0.1620 0.6199
PastTx     0.2057   0.0519  3.9646 0.0001  0.1040  0.3075 0.2621
PastType   0.0443   0.1109  0.3996 0.6895 -0.1731  0.2618 0.4324
AIntNo     0.0272   0.0528  0.5152 0.6064 -0.0763  0.1307 0.9139
AContNo   -0.1060   0.0651 -1.6284 0.1034 -0.2336  0.0216 0.9349
ATotalDur -0.0327   0.0098 -3.3470 0.0008 -0.0519 -0.0136 0.9675
================================================================

"""

In [247]:
np.exp(results.params[1:])

array([1.66552729, 0.99320365, 1.01185832, 0.97090898, 1.04401938,
       1.07259274, 1.2284412 , 1.04532712, 1.02756971, 0.89940697,
       0.96781559])